# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import csv
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("weatherpy.csv", sep=",")
data.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Ushuaia,-54.80,-68.30,39.20,60,40,21.92,AR
1,1,Thompson,55.74,-97.86,42.80,38,40,16.11,CA
2,2,Bredasdorp,-34.53,20.04,51.80,87,16,5.82,ZA
3,3,Sitka,57.05,-135.33,57.20,67,1,4.70,US
4,4,Leningradskiy,69.38,178.42,37.02,91,100,15.26,RU
5,5,Lompoc,34.64,-120.46,75.99,68,1,16.11,US
6,6,Saldanha,-33.01,17.94,55.40,87,46,2.24,ZA
7,7,Vaini,-21.20,-175.20,75.20,64,75,13.87,TO
8,8,Clyde River,70.47,-68.59,32.00,92,20,5.82,CA
9,9,Atuona,-9.80,-139.03,78.44,74,0,19.82,PF


In [3]:
data.dropna(inplace=True)
data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Ushuaia,-54.80,-68.30,39.20,60,40,21.92,AR
1,1,Thompson,55.74,-97.86,42.80,38,40,16.11,CA
2,2,Bredasdorp,-34.53,20.04,51.80,87,16,5.82,ZA
3,3,Sitka,57.05,-135.33,57.20,67,1,4.70,US
4,4,Leningradskiy,69.38,178.42,37.02,91,100,15.26,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = data[["Lat", "Lng"]].astype(float)
humidity = data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(data["Humidity"]),
                                point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#1st cond: 60 < temp < 70
#2nd cond: wind < 5
#3rd cond: no clouds
new_data = data.loc[(data["Max Temp"] < 70) & (data["Max Temp"] > 60) & (data["Wind Speed"] < 5) 
                    & (data["Cloudiness"] == 0)]

new_data = new_data.dropna()

new_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
140,140,Nurobod,40.91,69.80,60.80,59,0,4.70,UZ
200,200,Menzel Salem,35.86,8.48,64.40,63,0,2.24,TN
224,224,Bayir,37.27,28.22,69.80,83,0,3.09,TR
237,237,Aksu,41.12,80.26,63.75,34,0,1.81,CN
240,240,Shihezi,44.30,86.03,61.74,32,0,4.36,CN
276,276,Yangi Marg`ilon,40.43,71.72,62.60,51,0,4.70,UZ
281,281,Sorso,40.80,8.58,66.20,100,0,1.12,IT
346,346,Nevşehir,38.92,34.67,60.80,48,0,3.36,TR
358,358,Ampanihy,-24.70,44.75,65.75,87,0,4.90,MG
441,441,Constantine,36.37,6.61,64.40,68,0,2.24,DZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = new_data

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}...")
    
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # make request and print url
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    results = response["results"]

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]['name']
        hotel_df.loc[index, "City"] = response["results"][0]["City"]
        hotel_df.loc[index, "Country"] = response["results"][0]["Country"]
        print(f"Closest hotel is {results[0]['name']}.")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

Searching within Nurobod...
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
Searching within Menzel Salem...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.8540444,
                    "lng": 8.4746436
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.8556508302915,
                        "lng": 8.47714215
                    },
                    "southwest": {
                        "lat": 35.8529528697085,
                        "lng": 8.47381075
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "mohamed ben khlil omri",
            "place_id": "ChIJqSI2gGgz-hIRQS6xYayPwus",
            "plus_co

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAKjnzlLVzCanZw1GILHYMu9IsiNWmvrDTKjZV6dRRT5PN-_EDRQEmhtrwYoE8Se-7UACL8ytJWb6TxbH9M8EMOrZjmjbSM-iwzb-KjKmKi7BnPGB8qHJ-fJtpACjMUYMuj0OeLwFBSlp1__ecvR8PVlTddNMYvAt2eDX88V6jOp3ABiaQaHl3hKc6Jnc45NQQRa02XfMu7gfJau3TsjiFkuMN6rG4wyahmm-6vvfTPlVw76JtRKgl6e46BIpWLpTFx6rhWgg-X6KltC8Kg5HNNtlXWDkBT0oskOb1Io9nyWwfSrVKmcV-HT96OvsGYqzBxSCtRNPOENkZGNvGncv3vOSUYw9Cg3TV9SZHoWZHX6PwOaopoqzMCBrbpplafYU_xIQsaUp2sV1YK3WXbs2obnYxRoU_vkA0oiQKKSMUCoWLWb50E7PAK4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 44.296809,
                    "lng": 86.085748
                },
                "viewport": {
                    "northeast": {
                        "lat": 44.29818038029149,
                        "lng": 86.08709708029151
                    },
                    "southwest": {
                        "lat": 44.29548241970849,


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.4614177,
                    "lng": 71.7283138
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.4627439302915,
                        "lng": 71.7293732302915
                    },
                    "southwest": {
                        "lat": 40.4600459697085,
                        "lng": 71.7266752697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Ikathouse",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                        "<a href=\"https://maps.goo

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAHkr7L1-lMdXQACGGs3CqcMBzmNlP4ORmcZCNdCZt2-jFR0-TQV6rVssh-vvtDxPRJei_8382kiPVuIHJTEOFo0svhRlzbo5pq5zJETMxEIt57kUmLWSM_2aJhp_I2N_k1lmIwAgZ92tODTeOylBlcCnPSsTMx772G3bsvXcaLkrocD6o2VEIkAmm2VXcPDoUWcBN71Rv2GksYYevu4-H6OIcMTPbINOUwwcba5pwxxzypgxLx9z_e3rnx3RSKxeAOcA44Z4E_jz8zUEn37RoccGOLqsSP7JBH5lcOVMDW6G0VB86bA-6to4Qwf-H7pYmpNf84RQNTCXfXCaFMWgrb1H8y8tQ3SHqZ6IFGHzPJIW_-Oie68f_2tLVopUYXmzxRIQPha9zPf47Gz-x_KH8e9x1hoUbhUDM7AaXDgpKiZUyTU5cUWefDc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.797097,
                    "lng": 8.576625999999997
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.7984191302915,
                        "lng": 8.577996880291499
                    },
                    "southwest": {
                        "lat": 40.795721169

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -24.695365,
                    "lng": 44.74424
                },
                "viewport": {
                    "northeast": {
                        "lat": -24.6938044197085,
                        "lng": 44.74552223029151
                    },
                    "southwest": {
                        "lat": -24.6965023802915,
                        "lng": 44.74282426970851
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Angora Hotel",
            "photos": [
                {
                    "height": 3096,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111894283854262680694\">Jean Elys\u00e9e Louis</a>"
 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.2749686,
                    "lng": 47.5874935
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.27631238029149,
                        "lng": 47.58884073029149
                    },
                    "southwest": {
                        "lat": 34.2736144197085,
                        "lng": 47.58614276970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "\u0627\u06cc\u0632\u0648\u06af\u0627\u0645 \u0627\u0635\u0644 \u062f\u0644\u06cc\u062c\u0627\u0646 \u0648 \u0634\u0631\u0642",
            "photos": [
                {
                    "height": 683,
                    "html_attributions": [
             

In [9]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
140,140,Nurobod,40.91,69.80,60.80,59,0,4.70,UZ,NaN
200,200,Menzel Salem,35.86,8.48,64.40,63,0,2.24,TN,mohamed ben khlil omri
224,224,Bayir,37.27,28.22,69.80,83,0,3.09,TR,Buban Otel
237,237,Aksu,41.12,80.26,63.75,34,0,1.81,CN,Pudong Holiday Hotel
240,240,Shihezi,44.30,86.03,61.74,32,0,4.36,CN,Home Inn Shihezi Dongwu Road Century Square
276,276,Yangi Marg`ilon,40.43,71.72,62.60,51,0,4.70,UZ,Ikathouse
281,281,Sorso,40.80,8.58,66.20,100,0,1.12,IT,Eco's B&B
346,346,Nevşehir,38.92,34.67,60.80,48,0,3.36,TR,NaN
358,358,Ampanihy,-24.70,44.75,65.75,87,0,4.90,MG,Angora Hotel
441,441,Constantine,36.37,6.61,64.40,68,0,2.24,DZ,Grand Hotel Cirta


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color="green", stroke_color="navy",
    # the below line creates a list of strings to populate the info-box when you click on a datapoint on the map
    info_box_content=hotel_info)

# Create a mapping figure
fig = gmaps.figure()

# Add the layer to the figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))